**Задача**: оцифровка документов диагностики знаний обучающихся.  
На вход принимаются:  
* документы в формате `pdf`, содержание таблицы с результатами по заданиям, привязанным к шифрам диагностируемого  
* документы в формате `pdf`, содержание таблицы с соотнесением шифров с ФИО  

В *результате* работы скрипта: 
* отдельные excel-файлы результатов с ФИО  
* сводный excel-файл результатов с датами, предметами, ФИО, классами


Логика скрипта:  
* распаковка из zip-архивов (включая работу с кодировками)  
* извлечение информации из pdf и перевод в excel  
* сбор сводного датафрейма `df_joined`  

## Используемые библиотеки   
`os` — потребуется, чтобы работать с папками и файлами  
`zipfile` - потребуется, чтобы распаковывать zip-архивы  
`PyPDF2` - библиотека, которая позволяет работать с pdf-файлами. Нам важно, что информацию, записанную в файле как текст, можно перевести в единую строку текста. Внимание: нет OCR. Если страница сканированная, то без дополнительного распознавания работать не будет.    
`tabula` - потребуется, чтобы корректно переводить таблицы в pdf-файлах в DataFrame.

In [3]:
import os
import zipfile
import re
from PyPDF2 import PdfReader
import pandas as pd
import tabula
from tabula.io import read_pdf
import datetime

### Распаковка zip-файлов  
По условию все файлы изначально находятся в архива, поэтому распаковываем архивы и переназываем в "читаемой" кодировке файлы. 

Здесь и далее подразумевается, что есть корневая папка `path`, в которой находятся архивы с шифрами в `path/codes` (далее переменная `path_codes`) и `path/results` (далее переменная `path_results`) с шифрами и результатами диагностик соотвественно. 

In [5]:
path = 
for address, dirs, files in os.walk(path):
    for file in files:
        file_path = address +'/'+file
        if 'zip' in file_path:
            archive = zipfile.ZipFile(file_path)
            for entry in archive.infolist():
                encoding='cp866'
                encode = 'cp437'
                name = entry.filename.encode(encode).decode(encoding)
                archive.extract(entry.filename, address)
                os.rename(address + '/' + entry.filename, address + '/' + name)

### Собираем данные в таблицы    

С помощью `PdfReader` и функций `find_date` (`str_to_date`), `find_subject`, `find_distr`, `find_sch`, `find_cls` собираем дополнительные данные о дате, предмете, районе, школе и классе, к которому относится файл, после чего добавляем эти данные в таблицу результатов диагностик.

С помощью `tabula` выделяем результаты по каждому заданию и приводим названия столбцов к общему виду (*некоторые придётся учитывать в ручную*).  

Таблицы по шифрам и результатам сохранятся в excel в тех же папках, где и исходные файлы.

In [6]:
def str_to_date(date):
    dates_words = {'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 
                   'июня': 6, 'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12 }
    if '.' in date:
        dt = datetime.datetime.strptime(date, '%d.%m.%Y')
    elif '.' not in date and date != 'Не найдено':
        for word, initial in dates_words.items():
            date = date.replace(word.lower(), str(initial))                
        date = re.sub('-.*? ', ' ', date)
        dt = datetime.datetime.strptime(date, '%d %m %Y')
    else:
        dt = date
    return dt

In [7]:
def find_date (text):
    date = re.search(r'дата:\s?\d-?\d?\s?\D*\s?\d\d\d\d', text)
    #print(date[0] if date else 'Not match')
    date = date[0] if date else 'Не найдено'
    if date == 'Не найдено':
        date = re.search(r'дата: \d{1,2}.\d{1,2}.\d{2,4}', text)
        date = date[0] if date else 'Не найдено'
    date = re.sub('дата:\s?', '', date)
    return date

In [8]:
def find_subject (text):
    subject = re.search(r'предмет:\s?\S*\b', text)
    subject = subject[0] if subject else 'Не найдено'
    subject = re.sub(r'предмет:\s?', '', subject)
    return subject

In [9]:
def find_distr(text):
    distr = re.search(r'район:\s?\S*\b', text)
    distr = distr[0] if distr else 'Не найдено'
    distr = re.sub(r'район:\s?', '', distr)
    return distr

In [10]:
def find_sch(text):
    sch = re.search(r'школа:\s?.*класс', text)
    sch = sch[0] if sch else 'Не найдено'
    sch = re.sub(r'школа:\s?', '', sch)
    sch = re.sub(r'\s?класс', '', sch)
    return sch

In [11]:
def find_cls(text):
    cls = re.search(r'класс:\s?\S*\b', text)
    cls = cls[0] if cls else 'Не найдено'
    cls = re.sub(r'класс:\s?', '', cls)
    cls = re.sub(r'[\-_,#]', '', cls)
    return cls

In [12]:
for address, dirs, files in os.walk(path):
    for name in files:
        if 'pdf' not in name:
            pass
        else:
            file_path = os.path.join(address, name)
            reader = PdfReader(file_path)
            number_of_pages = len(reader.pages)
            page = reader.pages[0]
            text = page.extract_text().lower()
            
            for d in tabula.read_pdf(file_path, pages="all"):
                d.columns = [x.lower() for x in d.columns]
                d.columns = [re.sub('\r', ' ',x) for x in d.columns]
                d.rename(columns={'% выполнен ия': '% выполнения'}, inplace=True)
                d['дата'] = find_date(text)
                d['предмет'] = find_subject(text)
                d['район'] = find_distr(text)
                d['школа'] = find_sch(text)
                d['класс'] = find_cls(text)
                d.dropna(inplace=True)
                d.to_excel(os.path.join(address, str(name[0:-4]) +'.xlsx'), index=False)

### Сбор сводной таблицы  
Считываем все excel файлы (отдельно шифры и отдельно результаты). Создаём сводные таблицы по кодам и результатам (функция `pivot_table`).  

Объединяем данные по столбцам: `['шифр участника', 'предмет', 'класс']`. В нашем случае дата в файлах с шифрами не указывается, однако её также можно включить в столбцы `pd.merge`.

Сохраняем получившийся сводный файл в папке по пути `path_to_extract` с названием `pivot.xlsx`

In [14]:
def pivot_table (path):
    df = pd.DataFrame()
    for address, dirs, files in os.walk(path):
        for name in files:
            if 'xlsx' in name:
                df_read = pd.read_excel(os.path.join(address, name))
                df = pd.concat([df, df_read], ignore_index=True)
    return df

In [16]:
path_codes = 
path_results = 
path_to_extract = 

df_codes = pivot_table(path_codes)
df_results = pivot_table(path_results)
#print(df_codes, '\n',df_results)
df_codes = df_codes[['учащийся','шифр', 'предмет', 'класс']]
df_joined = pd.merge(df_results, df_codes, how='left', left_on=['шифр участника', 'предмет', 'класс'], right_on=['шифр', 'предмет', 'класс'])
cols = df_joined.columns.tolist()
cols = cols[0:1]+cols[-2:-1]+cols[1:-3]
df_joined = df_joined[cols]
df_joined.to_excel(os.path.join(path_to_extract + '/pivot.xlsx'), index=False)